# Early decoding

## Decoder-only

In [192]:
import torch
from transformers import AutoModelForCausalLM, DebertaForMaskedLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('gpt2-medium')
model = AutoModelForCausalLM.from_pretrained('gpt2-medium')

#model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")
#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

In [142]:
data = [{"context":  f'Q: What is the sentiment of "love"? A: positive Q: What is the sentiment of "hate"? A: negative',
           "prompt": f'Q: What is the sentiment of "{arg}"? A:', 
           "answ": ["positive", "negative"],
           "arg":  ["adore", 0]}]

In [196]:
def token_select(tokens, tokenizer, select_token="[MASK]"): #"Ġhi"
    if select_token is not None:
        select_token_id = tokenizer.convert_tokens_to_ids(select_token) ## retrieve index of [MASK]
        batch_idx, seq_idx = (tokens.input_ids == select_token_id).nonzero(as_tuple=True)
    
    if select_token is None or select_token not in tokenizer.vocab: ## get last token before padding
        batch_idx, seq_idx = (tokens.input_ids != tokenizer.pad_token_id).nonzero(as_tuple=True)
        batch_idx, unique_batch_counts = torch.unique_consecutive(batch_idx, return_counts=True)
        unique_batch_cumsum = torch.cumsum(unique_batch_counts,dim=0)-1
        seq_idx = seq_idx[unique_batch_cumsum]
    
    assert batch_idx.shape[0] > 0, f"mlm-type model and {select_token} token not in prompt text"
    batch_seq_idx = (batch_idx, seq_idx)
    return batch_seq_idx


def encode(texts, tokenizer, model):
    if tokenizer.pad_token is None: ## some tokenizers do not have pad tokens
        tokenizer.pad_token = tokenizer.eos_token  
    tokens = tokenizer(texts, padding=True, return_tensors='pt')
    output = model(**tokens,output_hidden_states=True)
    return output, tokens


def ids_to_tokens(scores, topk=5):
    pred_scores, pred_tokens = [], []
    topK_preds = torch.topk(scores, k=topk)
    for scores, indices in zip(topK_preds.values.tolist(), topK_preds.indices.tolist()):
        scores = list(map(lambda score: round(score,2), scores))
        pred_scores.append(scores)
        tokens = list(map(lambda idx: tokenizer.convert_ids_to_tokens(idx), indices))
        pred_tokens.append(tokens)
    return pred_tokens, pred_scores
    
    
def decode(h, model):
    if model.can_generate(): ## decoder-only
        scores = model.lm_head(h)
    else: ## encoder-only
        scores = model.cls(h)
    return scores

    
def early_decoding(hidden_states, tok_idx, model, l_start_end=[0,99]):
    for i, h in enumerate(hidden_states[l_start_end[0]:l_start_end[1]]):
        h = h[tok_idx] ## get hidden states per token
        scores = decode(h, model) ## decode the hidden state through last layer
        tokens, scores = ids_to_tokens(scores, topk=2)
        print(f"layer {i}: {list(zip(scores, tokens))}")

    
arg = "adore"
asw = ["positive", "negative"]
prompt = ["hi how are", "Summer is the"]

output, tokens = encode(prompt, tokenizer, model)
tok_idx = token_select(tokens, tokenizer)
early_decoding(output.hidden_states, tok_idx, model)

layer 0: [([5.33, 4.99], ['Ġare', 'Ġhelicop']), ([4.91, 4.27], ['Ġthe', 'Ġunden'])]
layer 1: [([18.36, 18.22], ['ĠUganda', 'ĠCitiz']), ([13.78, 10.66], ['ĠSponsor', 'Ġimperialist'])]
layer 2: [([13.67, 13.11], ['ĠUganda', 'Ġont']), ([8.19, 7.82], ['ĠUnited', 'Ġabove'])]
layer 3: [([12.04, 11.65], ['ĠUganda', 'ĠSponsor']), ([8.91, 8.38], ['Ġabove', 'ories'])]
layer 4: [([11.37, 11.3], ['ĠUganda', 'ĠSponsor']), ([9.12, 8.71], ['Ġbeginning', 'ories'])]
layer 5: [([6.49, 5.91], ['Ġdependencies', 'ĠUganda']), ([12.13, 9.53], ['Ġbeginning', 'Ġopposite'])]
layer 6: [([8.72, 7.95], ['ĠTours', 'ĠZen']), ([13.23, 12.63], ['Ġopposite', 'Ġbeginning'])]
layer 7: [([3.75, 2.42], ['ĠZen', 'Ġnot']), ([15.1, 14.33], ['Ġbeginning', 'Ġopposite'])]
layer 8: [([3.49, 2.24], ['ĠZen', 'Ġproceeding']), ([14.19, 13.38], ['Ġbeginning', 'Ġopposite'])]
layer 9: [([6.43, 4.01], ['Ġnot', 'Ġ[']), ([14.71, 10.55], ['Ġbeginning', 'Ġopposite'])]
layer 10: [([7.19, 4.9], ['Ġnot', 'Ġall']), ([15.29, 12.04], ['Ġbeginning'

In [59]:
#arg = "Canada"
#prompt = [f'On a scale from 0 to 10, the size of Andorra is a 0 and Russia is a 10.\
#The size of {arg} is a']

#arg = "Russia"
#prompt = [f'Compared to the size of the country Belgium, the size of {arg} is a']

arg = "adore"
asw = ["positive", "negative"]
prompt = [f'Q: What is the sentiment of "love"?\nA: positive\nQ: What is the sentiment of "hate"?\nA: negative\
          \nQ: What is the sentiment of "{arg}"?\nA:']

#arg = "Donald Trump and Joe Biden"
#prompt = [f'Q: What is the relationship between Trump and Biden? A: negative Q: What is the relationship between Romeo and Julia? A:']

#arg = "bad"
#prompt = [f'Q: What is the sentiment of "love"?\nA: 10\nQ: What is the sentiment of "hate"?\nA: 0\
#          \nQ: What is the sentiment of "{arg}"?\nA:']

#prompt = [f'Today I abandon. Yesterday I abandoned. Today I abolish. Yesterday I']
#prompt= ["Q: What is the capital of Peru? A: Lima Q: What is the capital of Germany? A:"]

token_input = tokenizer(prompt, return_tensors='pt')
output = model(**token_input,output_hidden_states=True)

In [3]:
def ids_to_tokens(scores, topk=5):
    topK_preds = torch.topk(scores, k=topk)
    pred_idx = topK_preds.indices[0].tolist()
    pred_scores = map(lambda score: round(score,2), topK_preds.values[0].tolist())
    pred_tokens = tokenizer.convert_ids_to_tokens(pred_idx)
    return pred_tokens, pred_scores
    
def decode(h):
    scores = model.lm_head(h)
    return scores
    
    
def early_decoding(output, l_start_end=[0,99]):
    hidden = output.hidden_states
    for i, h in enumerate(hidden[l_start_end[0]:l_start_end[1]]):
        h = h[...,-1,:] ## get logits for last token
        scores = decode(h)
        tokens, scores = ids_to_tokens(scores, topk=5)
        print(f"layer {i}: {list(zip(scores, tokens))}")
        #print(f"layer {i}: {tokens}")

#ids_to_tokens(output.logits[...,-1,:])
early_decoding(output)

layer 0: [(5.75, ':'), (4.7, ':"'), (4.67, ':['), (4.58, ':]'), (4.54, ':-')]
layer 1: [(23.16, 'ĠCitiz'), (22.47, 'ĠPetr'), (20.42, 'ifer'), (19.57, 'ĠDag'), (19.4, 'Ġappend')]
layer 2: [(24.85, 'ĠCitiz'), (24.05, 'ĠPetr'), (21.54, 'Ġont'), (21.51, 'Ġappend'), (21.5, 'ifer')]
layer 3: [(22.65, 'ĠPetr'), (20.98, 'ĠCitiz'), (20.97, 'Ġsubsc'), (20.71, 'Ġmum'), (20.57, 'Ġtion')]
layer 4: [(24.48, 'ĠPetr'), (22.5, 'Ġmum'), (21.24, 'ĠCitiz'), (20.88, 'ifer'), (20.68, 'ĠPutin')]
layer 5: [(24.9, 'ĠPetr'), (23.1, 'ĠGuinea'), (21.94, 'orate'), (21.38, 'Ġrevolutions'), (21.01, 'ĠAzerb')]
layer 6: [(25.51, 'mi'), (23.49, 'ĠPetr'), (23.24, 'orate'), (23.06, 'ĠGuinea'), (21.89, 'hene')]
layer 7: [(21.05, 'mi'), (17.92, 'ĠPlace'), (17.14, 'Ġrevolutions'), (17.03, 'Ġmum'), (16.77, 'ĠPetr')]
layer 8: [(17.41, 'Ġn'), (14.74, 'ĠPetr'), (14.69, 'Ġnil'), (13.62, 'mi'), (13.22, 'Ġnone')]
layer 9: [(17.23, 'Ġn'), (14.51, 'Ġnil'), (11.61, 'Ġinst'), (10.95, 'Ġtion'), (10.52, 'ĠPlace')]
layer 10: [(21.93, 'Ġn

## Encoder-only

In [186]:
tokenizer = AutoTokenizer.from_pretrained("lsanochkin/deberta-large-feedback")
model = DebertaForMaskedLM.from_pretrained("lsanochkin/deberta-large-feedback")

In [29]:
#test_adj = "Canada"
#prompt = [f'On a scale from 0 to 10, the size of Andorra is a 0 and Russia is a 10.\
#The size of {test_adj} is a']

#test_adj = "Russia"
#prompt = [f'Compared to the size of the country Belgium, the size of {test_adj} is a']

#test_adj = "bad"
#prompt = [f'Q: What is the sentiment of "love"?\nA: positive\nQ: What is the sentiment of "hate"?\nA: negative\
#          \nQ: What is the sentiment of "{test_adj}"?\nA:']

test_adj = "aweful"
prompt = [f'Q: What is the sentiment of "love"?\nA: 1\nQ: What is the sentiment of "hate"?\nA: 0\
          \nQ: What is the sentiment of "{test_adj}"?\nA: [MASK]']

#prompt = [f'Today I abandon. Yesterday I abandoned. Today I abolish. Yesterday I [MASK]']
#prompt= ["Q: What is the capital of Peru? A: Lima Q: What is the capital of Poland? A: [MASK]"]

tokens = tokenizer(prompt, return_tensors='pt')
output = model(**tokens,output_hidden_states=True)

In [30]:
def ids_to_tokens(scores, topk=5):
    topK_preds = torch.topk(scores, k=topk, dim=-1)
    pred_idx = topK_preds.indices[0].tolist()
    pred_scores = topK_preds.values[0].tolist()  
    if isinstance(pred_idx[0], list):
        pred_idx = pred_idx[0]
    pred_tokens = tokenizer.convert_ids_to_tokens(pred_idx)
    return pred_tokens
    
def decode(h):
    scores = model.cls(h)
    return scores
    
    
def early_decoding(output, tokens, l_start_end=[0,99], select_token="[MASK]"):
    hidden = output.hidden_states
    for i, h in enumerate(hidden[l_start_end[0]:l_start_end[1]]):
        
        select_token_id = tokenizer.convert_tokens_to_ids(select_token) ## retrieve index of [MASK]
        batch_idx, seq_idx = (tokens.input_ids == select_token_id).nonzero(as_tuple=True)  
        h = h[..., seq_idx,:]
        scores = decode(h)
        pred_tokens = ids_to_tokens(scores, topk=10)
        print(f"layer {i}: {pred_tokens}")


early_decoding(output, tokens)

tensor([55])
layer 0: ['x', ']', 'us', 'Ġadvance', 'Ġscale', 'Ġgiven', 'age', 'er', 'ade', 'Ġprocess']
tensor([55])
layer 1: ['ones', 'Ġadvance', 'Ġother', 'other', 'Ġfuture', 'Ġsurrounding', 'Ġchoke', 'Ġelementary', 'os', 'Ġsame']
tensor([55])
layer 2: ['Ġadvance', 'ones', 'Ġfuture', 'Ġelementary', 'Ġnew', 'Ġother', 'Ġold', 'Ġbehind', 'Ġday', 'Ġcommon']
tensor([55])
layer 3: ['Ġnew', 'Ġaddicted', 'Ġelementary', 'Ġother', 'other', 'Ġfuture', 'Ġold', 'Ġirresponsible', 'Ġof', 'Ġkeep']
tensor([55])
layer 4: ['Ġschool', 'Ġsocial', 'Ġnew', '.', 'Ġof', 'Ġday', 'Ġaddicted', 'Ġdistance', 'Ġamount', ':']
tensor([55])
layer 5: ['Ġaddicted', 'Ġthis', 'onymous', 'Ġonline', 'Ġdistance', 'Ġhome', 'Ġunderage', 'Ġfuture', 'Ġjust', 'Ġwhoever']
tensor([55])
layer 6: ['Ġwhoever', 'Ġthis', 'onymous', 'Ġdirty', 'ĠEveryday', 'Ġdistance', 'Ġonline', 'Ġdying', 'Ġhome', 'Ġunderage']
tensor([55])
layer 7: ['Ġonline', 'Ġthis', 'onymous', 'Ġfuture', 'ĠEveryday', 'ĠANY', 'Ġclassroom', 'Ġwhat', 'Ġhome', 'Ġoutgoing'

In [7]:
model.cls

DebertaOnlyMLMHead(
  (predictions): DebertaLMPredictionHead(
    (transform): DebertaPredictionHeadTransform(
      (dense): Linear(in_features=1024, out_features=1024, bias=True)
      (transform_act_fn): GELUActivation()
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
    )
    (decoder): Linear(in_features=1024, out_features=50265, bias=True)
  )
)

In [8]:
select_token_id = tokenizer.convert_tokens_to_ids(select_token)  ## retrieve index of [MASK]
batch_idx, select_token_idx = (tokens.input_ids == select_token_id).nonzero(as_tuple=True)

NameError: name 'select_token' is not defined